![Deep Learning for Scientists in a hurry](./fig/Title.png)

In [ ]:
%load_ext watermark

In [ ]:
%watermark

In [ ]:
import os
import time
start = time.time()
chapter_number = 1
import matplotlib
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
from torchvision import transforms
from PIL import Image
import torch

In [ ]:
%watermark -iv

In [ ]:
os.environ["TORCH_HOME"] = "./data"

---

# Using pre-trained networks (AlexNet and ResNet101)

Before learning how to build Deep Neural Networks, let's start an exploration with one of the most iconic networks, already trained and ready to be used, AlexNet.

AlexNet was the network that won the ImageNet Large Scale Visual Recognition Competition (ILSVRC) and revolutionized Deep Learning.

The paper is titled [ImageNet classification with deep convolutional neural networks](https://doi.org/10.1145/3065386) and the authors were Alex Krizhevsky, Ilya Sutskever, and Geoffrey E. Hinton.

![AlexNet1](./fig/AlexNet1.png)

![AlexNet2](./fig/AlexNet2.png)


ImageNet is a very large dataset with more than 14 million images and it is maintained by Stanford University.
Images on the dataset receive labels with a hierarchy of nouns coming from the [WordNet dataset](https://wordnet.princeton.edu). WordNet is a large list of words in the English language. WordNet contains nouns, verbs, adjectives, and adverbs and they are grouped into sets of cognitive synonyms (synsets), each expressing a distinct concept. 



AlexNet was trained with a subset of the ImageNet dataset consisting on 1.2 million images labeled with one in 1000 nouns.

The result is a **function**, ie a map that receives one image and returns the probability for that image to receive each of the 1000 labels. The biggest value will be associated with most probable label the image will receive.

## The models available in PyTorch

PyTorch offers many Networks ready to use. The list is quite impressive:

In [ ]:
index = 0
for iname in dir(models):
    if not iname.startswith('_'):
        print("%30s" % iname, end='')
        index += 1
    if index % 3 == 0:
        print('')

## Selecting `AlexNet`

We will choose `AlexNet` and download the pre-trained version of the network. The network is big so it takes a while. 

What we are downloading is not only the network but also all the weights and biases, including those for the convolutions that make the network perform as it was trained.

In [ ]:
alexnet = models.AlexNet()
alexnet

See the definition of this network and try to compare it with the figure above. Notice the numbers $11 \times 11$. Not all the numbers match and this network is quite complex. We will understand it much better later on.

In [ ]:
!ls ./data/hub/checkpoints

In [ ]:
import torchvision

In [ ]:
major,minor = torchvision.version.__version__.split('.')[0:2]
major=int(major)
minor=int(minor)

In [ ]:
if major==0 and minor < 13:
    alexnet = models.alexnet(pretrained=False)
else:
    alexnet = models.alexnet(weights=None)

weights = torch.load("./data/hub/checkpoints/alexnet-owt-4df8aa71.pth")

In [ ]:
alexnet.load_state_dict(weights)

## ResNet-101

ResNet-101 is another network from [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf). This network won the ImageNet classification, detection, and localization challenge in 2015.

ResNet is a family of powerful deep neural networks which has achieved remarkable performance results in the ILSVRC 2015 classification challenge. 
ResNet has outperformed generalization performance on other recognition tasks and won first place on ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation in ILSVRC and COCO 2015 competitions. 

There are many variants of ResNet architecture i.e. same concept but with a different number of layers. 
We have ResNet-X where X indicates the number of layers. X could be 18, 34, 50, 101,152, 164, 1202

The name ResNet followed by a two or three-digit number simply implies the ResNet architecture with a certain number of neural network layers. 

|Residual Network Weights | Accuracy at 1 | Accuracy in first 5 | Number of params |
|---|---|---|---|
|ResNet18_Weights.IMAGENET1K_V1  | 69.758 | 89.078 | 11.7M |
|ResNet34_Weights.IMAGENET1K_V1  | 73.314 | 91.420 | 21.8M |
|ResNet50_Weights.IMAGENET1K_V1  | 76.130 | 92.862 | 25.6M |
|ResNet50_Weights.IMAGENET1K_V2  | 80.858 | 95.434 | 25.6M |
|ResNet101_Weights.IMAGENET1K_V1 | 77.374 | 93.546 | 44.5M |
|ResNet101_Weights.IMAGENET1K_V2 | 81.886 | 95.780 | 44.5M |
|ResNet152_Weights.IMAGENET1K_V1 | 78.312 | 94.046 | 60.2M |
|ResNet152_Weights.IMAGENET1K_V2 | 82.284 | 96.002 | 60.2M |


In [ ]:
if major==0 and minor < 13:
    resnet_18 = models.resnet18(pretrained=False)
    resnet_34 = models.resnet34(pretrained=False)
    resnet_50 = models.resnet50(pretrained=False)
    resnet_101 = models.resnet101(pretrained=False)
    resnet_152 = models.resnet152(pretrained=False)
else:
    resnet_18 = models.resnet18(weights=None)
    resnet_34 = models.resnet34(weights=None)
    resnet_50 = models.resnet50(weights=None)
    resnet_101 = models.resnet101(weights=None)
    resnet_152 = models.resnet152(weights=None)

weights = torch.load("./data/hub/checkpoints/resnet101-5d3b4d8f.pth")
resnet_101.load_state_dict(weights)

The network is quite complex. See the image above from the paper to get an idea of all the layers designed on the network.

In [ ]:
resnet = resnet_101

## Loading an image

Let's start with one image of a Box Turtle, a beautiful turtle native to North America.

In [ ]:
img = Image.open("./data/box-turtle-1389243_1920-740x500.jpg")
img

## Preprocessing the image

Images can have many sizes, a number of pixels in height and width, however, our network uses a very specific size $224 \times 224$.

This size is convenient because when using a $7 \times 7$ kernel we will get 32 blocks and the network continues in deeper layers using those values. 

Another point is that images use an integer number, 3 numbers in fact for each of the 3 colors; Red, Green, and Blue (RGB). Neural networks usually work better with small positive real numbers.

All this calls for a preprocessing of the image and we do that with `transforms`:

In [ ]:
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

Using the preprocess object, we offer the image as an argument:

In [ ]:
img_tensor = preprocess(img)
img_tensor

We have a PyTorch tensor witch shape:

In [ ]:
img_tensor.shape

This tensor has $3 \times 224 \times 224$, that is one grid for each color. We can use the normalized tensor to plot image decomposed.

In [ ]:
fig,axs = plt.subplots(1,3, sharey=True, figsize=(12,6))
axs[0].imshow(img_tensor[0],cmap='Reds')
axs[1].imshow(img_tensor[1],cmap='Greens')
axs[2].imshow(img_tensor[2],cmap='Blues');

## Creating a batch 

Models work with batches of inputs. We have just one image so we will create a batch from an image.
The method `unsqueeze` will add an extra dimension at the beginning of the array.

In [ ]:
batch_tensor = torch.unsqueeze(img_tensor, 0)

In [ ]:
batch_tensor.shape

In the PyTorch convention the input must follow the schema:
    
$\text{batch size} \times \text{channels} \times \text{pixel width} \times \text{pixels height}$

We have prepared the right tensor that we can feed for model evaluation

## Model Evaluation

Let's use resnet evaluated with the trained values

In [ ]:
resnet.eval()

The `resnet` works like a function that receives as argument a batch and returns an output tensor.

In [ ]:
out = resnet(batch_tensor)
out

In [ ]:
out.shape

This tensor contains unnormalized values for all the 1000 classes returned by `resnet`.

## Loading labels

In [ ]:
with open('./data/imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

In [ ]:
len(labels)

## Identifying the image

In [ ]:
_, index = torch.max(out, 1)

In [ ]:
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
labels[index[0]], percentage[index[0]].item()

In [ ]:
_, indices = torch.sort(out, descending=True)

In [ ]:
for idx in indices[0][:10]:
    print("{name:70s} Probability: {prob:8.5f}".format(name=labels[idx], prob=percentage[idx].item()))

# Exercises

1. Upload your image of an animal. Put the image on the "images" folder and execute the steps to identify the image using the pre-trained ResNet-101.

2. Search for an image with two animals, like a cat and a dog. What is the answer to the network?

---

# References

There are many books about Deep Learning and many more on Machine Learning. 
This list is by no means an exhaustive list of books. I am listing the books from which I took inspiration. Also, I am listing materials where I found better ways to present topics. Often I am amazed by how people can create approachable materials for seemingly dry subjects.

The order of the books goes from divulgation and practical to the more rigorous and mathematical. Slides, blogs, and videos are those I have found over the internet or suggested by others.

### Selection of Books on Deep Learning

<br>
<div style="clear: both; display: table;">
  <div style="border: none; float: left; width: 200; padding: 5px">
  <img alt="Deep Learning - Kelleher" 
       src="./fig/books/Deep Learning - Kelleher.jpg" 
       height="100" width="100"  />
  </div>
  <div style="border: none; float: left; width: 800; padding: 5px">
      Deep Learning<br>
      John D. Kelleher<br>
      2019<br>
  </div>
</div>

<br>
<div style="clear: both; display: table;">
  <div style="border: none; float: left; width: 200; padding: 5px">
  <img alt="Introduction to Deep Learning - Charniak" 
       src="./fig/books/Introduction to Deep Learning - Charniak.jpg" 
       height="100" width="100"  />
  </div>
  <div style="border: none; float: left; width: 800; padding: 5px">
      Introduction to Deep Learning<br>
      Eugene Charniak<br>
      2018<br>
  </div>
</div>

<br>
<div style="clear: both; display: table;">
  <div style="border: none; float: left; width: 200; padding: 5px">
  <img alt="Introduction to Deep Learning - Skansi" 
       src="./fig/books/Introduction to Deep Learning - Skansi.jpg" 
       height="100" width="100"  />
  </div>
  <div style="border: none; float: left; width: 800; padding: 5px">
      Introduction to Deep Learning<br>
      Sandro Skansi<br>
      2018<br>
  </div>
</div>

<br>
<div style="clear: both; display: table;">
  <div style="border: none; float: left; width: 200; padding: 5px">
  <img alt="Deep Learning with PyTorch - Subramanian" 
       src="./fig/books/Deep Learning with PyTorch - Subramanian.jpg" 
       height="100" width="100"  />
  </div>
  <div style="border: none; float: left; width: 800; padding: 5px">
      Deep Learning with PyTorch<br>
      Vishnu Subramanian<br>
      2018<br>
  </div>
</div>

<br>
<div style="clear: both; display: table;">
  <div style="border: none; float: left; width: 200; padding: 5px">
  <img alt="Deep Learning with PyTorch - Stevens" 
       src="./fig/books/Deep Learning with PyTorch - Stevens.png" 
       height="100" width="100"  />
  </div>
  <div style="border: none; float: left; width: 800; padding: 5px">
      Deep Learning with PyTorch<br>
      Eli Stevens, Luca Artiga and Thomas Viehmann<br>
      2020<br>
  </div>
</div>

<br>
<div style="clear: both; display: table;">
  <div style="border: none; float: left; width: 200; padding: 5px">
  <img alt="Deep Learning with Python - Chollet" 
       src="./fig/books/Deep Learning with Python - Chollet.jpg" 
       height="100" width="100" />
  </div>
  <div style="border: none; float: left; width: 800; padding: 5px">
      Deep Learning with Python (Second Edition)<br>
      François Chollet<br>
      2021<br>
  </div>
</div>

<br>
<div style="clear: both; display: table;">
  <div style="border: none; float: left; width: 200; padding: 5px">
  <img alt="Deep Learning - Patterson" 
       src="./fig/books/Deep Learning - Patterson.jpeg"
       height="100" width="100" />
  </div>
  <div style="border: none; float: left; width: 800; padding: 5px">
      Deep Learning, a practitioner's approach<br>
      Josh Patterson and Adam Gibson<br>
      2017<br>
  </div>
</div>

<br>
<div style="clear: both; display: table;">
  <div style="border: none; float: left; width: 200; padding: 5px">
  <img alt="Deep Learning - Goodfellow" 
       src="./fig/books/Deep Learning - Goodfellow.jpg" 
       height="100" width="100"  />
  </div>
  <div style="border: none; float: left; width: 800; padding: 5px">
      Deep Learning<br>
      Ian Goodfellow, Yoshua Bengio, and Aaron Courville<br>
      2016<br>
  </div>
</div>

### Interactive Books

  * [Dive into Deep Learning](https://d2l.ai/index.html)<br>
    Interactive deep learning book with code, math, and discussions<br> 
    Implemented with PyTorch, NumPy/MXNet, and TensorFlow<br>
    Adopted at 300 universities from 55 countries


### Slides

  * John Urbanic's ["Deep Learning in one Afternoon"](https://www.psc.edu/wp-content/uploads/2022/04/Deep-Learning.pdf)<br>
An excellent fast, condensed introduction to Deep Learning.<br>
John is a Parallel Computing Scientist at Pittsburgh Supercomputing Center

  * [Christopher Olah's Blog](http://colah.github.io) is very good. For example about [Back Propagation](http://colah.github.io/posts/2015-08-Backprop)

  * Adam W. Harley on his CMU page offers [An Interactive Node-Link Visualization of Convolutional Neural Networks](https://www.cs.cmu.edu/~aharley/vis/)



### Jupyter Notebooks

 * [Yale Digital Humanities Lab](https://github.com/YaleDHLab/lab-workshops)
 
 * Aurelien Geron Hands-on Machine Learning with Scikit-learn 
   [First Edition](https://github.com/ageron/handson-ml) and
   [Second Edition](https://github.com/ageron/handson-ml2)
   
 * [A progressive collection notebooks of the Machine Learning course by the University of Turin](https://github.com/rugantio/MachineLearningCourse)
   
 * [A curated set of jupyter notebooks about many topics](https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks)
   
### Videos

 * [Caltech's "Learning from Data" by Professor Yaser Abu-Mostafa](https://work.caltech.edu/telecourse.html)
 
 * [3Blue1Brown Youtube Channel](https://www.youtube.com/watch?v=Ilg3gGewQ5U)
 
 ---

# Back of the Book

In [ ]:
n = chapter_number
t = np.linspace(0, (2*(n-1)+1)*np.pi/2, 1000)
x = t*np.cos(t)**3
y = 9*t*np.sqrt(np.abs(np.cos(t))) + t*np.sin(0.3*t)*np.cos(2*t)
plt.plot(x, y, c="green")
plt.axis('off');

In [ ]:
end = time.time()
print(f'Chapter {chapter_number} took {int(end - start):d} seconds')